# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Beuther  ->  H. Beuther  |  ['H. Beuther']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 60 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.10128


extracting tarball to tmp_2508.10128... done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']


Found 82 bibliographic references in tmp_2508.10128/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.10329


extracting tarball to tmp_2508.10329... done.
Retrieving document from  https://arxiv.org/e-print/2508.10895


extracting tarball to tmp_2508.10895...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


Found 112 bibliographic references in tmp_2508.10895/main-v3.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.10128-b31b1b.svg)](https://arxiv.org/abs/2508.10128) | **Pinched Magnetic Fields in the High-mass Protocluster W3 IRS5**  |
|| H.-R. V. Chen, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-08-15*|
|*Comments*| *19 pages, 7 figures, accepted for publication in The Astrophysical Journal*|
|**Abstract**|            We present polarization maps of dust emission at 340 GHz in the luminous high-mass protocluster, W3 IRS5, observed with the Submillimeter Array. The projected magnetic fields appear fairly organized with a pinched morphology in the northern part and a concave shape in the southern part. We fit the polarization maps with a two-component magnetic field model: an hourglass model centered at the continuum peak, SMM2, and an empirical sphere centered at the O-type star, IRS7. Using the Davis-Chandrasekhar-Fermi method, we calculate a projected field strength of $B_\mathrm{pos} = 1.4 \; \mathrm{mG}$. Along with the Zeeman measurement, a total magnetic field strength of $B_\mathrm{tot} = 1.6 \; \mathrm{mG}$ is obtained. We find that the gravitational energy is the most dominant, followed by magnetic energy, and then turbulent energy. Small values of the virial parameter, $\alpha_\mathrm{vir} = 0.8$, and the ratio of timescales, $t_\mathrm{ff}/t_\mathrm{corss} = 0.6$, suggest an ongoing collapse. We also show collimated molecular outflows in the $\mathrm{CO \; (3-2)}$ and $\mathrm{SiO \; (8-7)}$ transitions. The morphology of magnetic fields and the surrounding \HII regions put forward a scenario for W3 IRS5. A gravitationally unstable dense core formed within a neutral gas ridge plowed by the expansions of W3 A and W3 B. The core began to contract, causing gravity to pull the magnetic field lines inward, which resulted in a pinched field morphology. Subsequent expansion of W3 F, ionized by IRS7, perturbed the magnetic field, creating concave patterns. The dynamical interactions among protostars led to misalignment of their outflows.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.10895-b31b1b.svg)](https://arxiv.org/abs/2508.10895) | **Stars Born in the Wind: M82's Outflow and Halo Star Formation**  |
|| V. V. Rao, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-08-15*|
|*Comments*| *22 pages; 11 figures; 1 table; accepted for publication in The Astrophysical Journal*|
|**Abstract**|            Starburst galaxies, like M82, launch kiloparsec-scale galactic outflows that interact with the circumgalactic medium (CGM) in complex ways. Apart from enriching the CGM with metals and energy, these outflows may trigger star formation in the halo -- either by driving shocks into the CGM or transporting cold, star-forming gas. To investigate such processes, we analyze the star formation history (SFH) of the Southern Arcs -- arc-like stellar features located ~5 kpc from M82's star-forming disk along the minor axis -- using Hubble Space Telescope Wide Field Camera 3 photometry. From resolved stellar populations, we derive SFHs over the last ~500 Myr, finding that ~85% of the stellar mass formed between ~150 and ~70 Myr ago, followed by a brief pause, with the remaining ~15% forming since ~30 Myr ago. The two stellar populations are co-spatial on scales of at least ~200 pc. The timing of the ~100 Myr burst aligns with star formation in the M82 disk and the age distribution of its star clusters, suggesting a causal link between the disk starburst and halo star formation. We explore two mechanisms that could explain these observations. In the first, shocks driven by the interaction between hot outflowing gas and cooler CGM material compress dense clouds, triggering collapse and star formation. In the second, stars form directly within massive, cool clouds associated with the outflow. As these clouds move ballistically through the halo, subsequent interactions with tidal debris may trigger additional star formation, producing the observed episodic structure.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.10329-b31b1b.svg)](https://arxiv.org/abs/2508.10329) | **Toward High-Precision Astrometry with CSST Using Multi-Gaussian Fitting of PSF**  |
|| J. Nie, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-08-15*|
|*Comments*| **|
|**Abstract**|            The Chinese Space Station Survey Telescope (CSST) presents significant potential for high-precision astrometry. In this study, we show that the point spread function (PSF) modeled by the discrete PSF with Multi-Gaussian function can effectively enhance the astrometric accuracy. We determine that the PSF profile can be accurately modeled by three Gaussians, which takes advantage of reduced computational complexity in PSF convolution. In sparse star fields, the lowest centering accuracy we obtain after aberration correction can be below 1 mas. We find that the proper motion errors remain below 1.0 mas/yr for point sources with five observations and approximately 0.8 mas/yr for seven observations with a time baseline of around 3.5 years. We finally demonstrate that the precision of our position measurements for stars fainter than 21 mag in the simulated CSST crowded field is better than the results from both SExtractor and DOLPHOT.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.10128/./f1.png', 'tmp_2508.10128/./f4.png', 'tmp_2508.10128/./f5.png']
copying  tmp_2508.10128/./f1.png to _build/html/
copying  tmp_2508.10128/./f4.png to _build/html/
copying  tmp_2508.10128/./f5.png to _build/html/
exported in  _build/html/2508.10128.md
    + _build/html/tmp_2508.10128/./f1.png
    + _build/html/tmp_2508.10128/./f4.png
    + _build/html/tmp_2508.10128/./f5.png
found figures ['tmp_2508.10895/./spatial-dist.png', 'tmp_2508.10895/./age-compare.png', 'tmp_2508.10895/./m82-angst.png']
copying  tmp_2508.10895/./spatial-dist.png to _build/html/
copying  tmp_2508.10895/./age-compare.png to _build/html/
copying  tmp_2508.10895/./m82-angst.png to _build/html/
exported in  _build/html/2508.10895.md
    + _build/html/tmp_2508.10895/./spatial-dist.png
    + _build/html/tmp_2508.10895/./age-compare.png
    + _build/html/tmp_2508.10895/./m82-angst.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HI}{\mbox{H\thinspace{\sc i} }}$
$\newcommand{\HII}{\mbox{H\thinspace{\sc ii} }}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\dif}{\mathrm{d}}$
$\newcommand{\key}[1]{{\color{magenta}\bf #1}}$
$\newcommand{\vec}{\boldsymbol}$</div>



<div id="title">

# Pinched Magnetic Fields in the High-mass Protocluster W3 IRS5

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.10128-b31b1b.svg)](https://arxiv.org/abs/2508.10128)<mark>Appeared on: 2025-08-15</mark> -  _19 pages, 7 figures, accepted for publication in The Astrophysical Journal_

</div>
<div id="authors">

H.-R. V. Chen, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** We present polarization maps of dust emission at 340 GHz in the luminous high-mass protocluster, W3 IRS5, observed with the Submillimeter Array.The projected magnetic fields appear fairly organized with a pinched morphology in the northern part and a concave shape in the southern part.We fit the polarization maps with a two-component magnetic field model: an hourglass model centered at the continuum peak, SMM2, and an empirical sphere centered at the O-type star, IRS7.Using the Davis-Chandrasekhar-Fermi method, we calculate a projected field strength of $B_\mathrm{pos} = 1.4 \; \mathrm{mG}$ .Along with the Zeeman measurement, a total magnetic field strength of $B_\mathrm{tot} = 1.6 \; \mathrm{mG}$ is obtained.We find that the gravitational energy is the most dominant, followed by magnetic energy, and then turbulent energy.Small values of the virial parameter, $\alpha_\mathrm{vir} = 0.8$ , and the ratio of timescales, $t_\mathrm{ff}/t_\mathrm{corss} = 0.6$ , suggest an ongoing collapse.We also show collimated molecular outflows in the $\mathrm{CO \; (3-2)}$ and $\mathrm{SiO \; (8-7)}$ transitions.The morphology of magnetic fields and the surrounding $\HII$ regions put forward a scenario for W3 IRS5.A gravitationally unstable dense core formed within a neutral gas ridge plowed by the expansions of W3 A and W3 B.The core began to contract, causing gravity to pull the magnetic field lines inward, which resulted in a pinched field morphology.Subsequent expansion of W3 F, ionized by IRS7, perturbed the magnetic field, creating concave patterns.The dynamical interactions among protostars led to misalignment of their outflows.

</div>

<div id="div_fig1">

<img src="tmp_2508.10128/./f1.png" alt="Fig1" width="100%"/>

**Figure 1. -** (a) SMA 340 GHz continuum map (color scale and red contours) of W3 IRS5 overlaid on VLA $6 \; \mathrm{cm}$ continuum map \citep[gray contours; ][]{Tieftrunk1997} with labels of the nearby $\HI$I regions, W3 A, W3 B, and W3 F.
Red contours are plotted at $(-3,3,6,14,22,30,38,46,54,62) \times \sigma$, where $\sigma = 21 \; \mathrm{mJy   beam^{-1}}$ with a beam size of $2$\farcs$7 \times 2$\farcs$2$($\mathrm{P. A.} = 3^\circ$).
Blue crosses mark the positions of five compact continuum sources identified at $354 \; \mathrm{GHz}$ ([Wang, Bourke and Hogerheijde 2013]()) , where SMM2 coincides with the continuum peak, and SMM1 is located to the east.
Green open circles are the positions of near-infrared sources  ([Megeath, Wilson and Corbin 2005]()) , and green filled squares are the positions of compact radio sources at $22 \; \mathrm{GHz}$ and $43 \; \mathrm{GHz}$ ([Tuthill and Danchi 2005]()) .
The blue-filled star marks the position of the high-mass star, IRS7, powering the hypercompact $\HI$I region, W3 F, while another infrared source, IRS N7 (blue-open star), without any radio counterpart, is also shown  ([Bik, Henning and Stolte 2012]()) .
Several outflow axes (and a bipolar outflow along line of sight) reported in the literature are presented with labels of their associated tracers, including molecular outflows \citep[e.g.][]{Mitchell1991,Rodon2008,Wang2012}, water masers  ([Imai, Kameya and Sasao 2000]()) , and radio compact sources  ([Wilson, et. al 2003]()) .
The gray dashed contours outline the more extended $6 \; \mathrm{cm}$ radiation at $4 \; \mathrm{mJy   beam^{-1}}$ with $3^{\prime\prime}$ resolution.
(b) The SMA polarized intensity map (color scale) overlaid with the orientation of the projected magnetic field, $B_\mathrm{pos}$(line segment), which corresponds to dust polarization orientation rotated by $90^\circ$.
Line segments are sampled twice per synthesized beam in locations where $I > 3\sigma_I$ and $P > 3\sigma_P$(in black), and also $2\sigma_P < P < 3\sigma_P$(in purple).
The length of line segments is proportional to the fractional linear polarization in percentage, $P/I$, and the scalebar is shown in the upper right.
Three thick line segments in gray show the large-scale magnetic field orientation measured from single-dish polarimeters.
Red and blue arrows indicate bipolar outflows along with the outflow label.
 (*fig:bvec*)

</div>
<div id="div_fig2">

<img src="tmp_2508.10128/./f4.png" alt="Fig4" width="100%"/>

**Figure 4. -** (a) Best-fit magnetic field model (cyan) overlaid on the observed magnetic field orientation (black) for models composed of an hourglass component centered at SMM2 combined with a spherical component centered at IRS 7.
Data on the outskirts are excluded for optimization.
The length of line segments is fixed and arbitrary.
Red and blue arrows indicate bipolar outflows, same as the outflow axes shown in Fig. \ref{fig:bvec}(a).
Symbols are the same as those in Fig. \ref{fig:bvec}(b).
(b)--(e) Histogram of anglar distribution for (b) observed position angle, $\psi_\mathrm{obs}$, (c) measurement uncertainty, $\sigma_{\psi_\mathrm{obs}}$, (d) position angle of the best-fit model, $\psi_\mathrm{mod}$, and (e) field oritation residual, $\Delta \psi_\mathrm{obs} = \psi_\mathrm{obs} - \psi_\mathrm{mod}$.
 (*fig:bmod*)

</div>
<div id="div_fig3">

<img src="tmp_2508.10128/./f5.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
Polarization maps of the (a) Stokes-$I$, (b) Stokes-$Q$, and (c) Stokes-$U$ parameters.
The thick gray contour outlines the $340 \; \mathrm{GHz}$ continuum emission at the $3\sigma$ level.
 (*fig:iqu*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.10128"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Stars Born in the Wind: M82’s Outflow and Halo Star Formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.10895-b31b1b.svg)](https://arxiv.org/abs/2508.10895)<mark>Appeared on: 2025-08-15</mark> -  _22 pages; 11 figures; 1 table; accepted for publication in The Astrophysical Journal_

</div>
<div id="authors">

V. V. Rao, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Starburst galaxies, like M82, launch kiloparsec-scale galactic outflows that interact with the circumgalactic medium (CGM) in complex ways. Apart from enriching the CGM with metals and energy, these outflows may trigger star formation in the halo- either by driving shocks into the CGM or transporting cold, star-forming gas. To investigate such processes, we analyze the star formation history (SFH) of the Southern Arcs—arc-like stellar features located $\sim5$ kpc from M82’s star-forming disk along the minor axis—using _Hubble Space Telescope_ Wide Field Camera 3 photometry. From resolved stellar populations, we derive SFHs over the last $\sim500$ Myr, finding that $\sim85\%$ of the stellar mass formed between $\sim150$ and $\sim70$ Myr ago, followed by a brief pause, with the remaining $\sim15$ \% forming since $\sim30$ Myr ago. The two stellar populations are co-spatial on scales of at least $\sim200 $ pc. The timing of the $\sim100$ Myr burst aligns with star formation in the M82 disk and the age distribution of its star clusters, suggesting a causal link between the disk starburst and halo star formation. We explore two mechanisms that could explain these observations. In the first, shocks driven by the interaction between hot outflowing gas and cooler CGM material compress dense clouds, triggering collapse and star formation. In the second, stars form directly within massive, cool clouds associated with the outflow. As these clouds move ballistically through the halo, subsequent interactions with tidal debris may trigger additional star formation, producing the observed episodic structure.

</div>

<div id="div_fig1">

<img src="tmp_2508.10895/./spatial-dist.png" alt="Fig6" width="100%"/>

**Figure 6. -** (**Left**) CMD selection of $\sim10$ Myr and $\sim100$ Myr stars from model Hess diagrams generated from PADUA models. (**Top Right**) Spatial distribution of $\sim10$ Myr stars (blue) and $\sim100$ Myr stars (orange). (**Middle Right**) 2D spatial cross-correlation between the $\sim10 $Myr and $\sim100 $Myr populations. Pixels that correspond to shifts along the direction of the outflow and having a correlation $>0.5$ are marked in green. (**Bottom Right**) RGB stars from the ancient halo population appear randomly distributed. (*fig:spat-dist*)

</div>
<div id="div_fig2">

<img src="tmp_2508.10895/./age-compare.png" alt="Fig7" width="100%"/>

**Figure 7. -** The SFH of the Southern Arcs appears correlated with the SFH of the central M82 disk. (**Left**) Resolved stars in the M82 field from ANGST and the Southern Arcs. Regions for which SFHs were calculated by [Williams, Dalcanton and Johnson (2011)]()(POS2 and POS5) are shown in the same plot. Locations of young massive star clusters (YMCs) studied by [Li, Anders and Li (2015)]() are also shown for reference. (**Top Right**) SFHs of POS2 (cyan) and POS5 (orange) scaled down by a factor of 20 compared to the SFH of the Southern Arcs obtained from PADUA models (blue). POS2 and POS5 are dominated by stars from the central disk region of M82. The Southern Arcs and the central M82 disk both experienced enhanced star formation $\sim100$ Myr ago. (**Bottom Right**) Histogram of star cluster ages from [Li, Anders and Li (2015)]()(pink) compared to the SFH of the Southern Arcs obtained from PADUA models (blue). The M82 disk experienced multiple epochs of star cluster formation similar to the Southern Arcs. (*fig:age-compare*)

</div>
<div id="div_fig3">

<img src="tmp_2508.10895/./m82-angst.png" alt="Fig3" width="100%"/>

**Figure 3. -** (**Left**) Resolved stars in the M82 field from ANGST  ([Dalcanton, Williams and Seth 2009]())  with the stars from the Southern Arcs field (this work). The points in cyan represent populations of stars $\lesssim 400$ Myr, selected by excluding the red giant branch (RGB) stars from the color-magnitude diagram (CMD) of each dataset (dashed grey regions in the panels on the right) with matched depths (F814W$<26$). The M82 disk has been masked out for these stars to accentuate structures that follow the breakout of the outflow from the disk. Arc-like features in the Southern Arcs field appear to be spatial extensions of young stellar structures in ANGST that may be related to M82's starburst outflow. (**Top Right**) F814W vs. F435W-F814 CMD of `good star' (GST) sources from ANGST with some typical PARSEC  ([Bressan, Marigo and Girardi 2012]())  isochrones (shifted assuming $A_V=0.1$ and $(m-M)_0 =27.8$). The 10 Gyr isochrones have metallicities, [M/H] = 0, -0.5, -1.0, -1.5, -2.0. The younger isochrones have [M/H] = 0.
    (**Bottom Right**) CMD of `GST' sources from the Southern Arcs. PARSEC isochrones  with a foreground extinction (shifted assuming $A_V=0.35$ and $(m-M)_0=27.8$) are over-plotted to broadly explain the multiple stellar populations. The 10 Gyr isochrones ([M/H] = 0, -0.5, -1.0, -1.5, -2.0) correspond to the old, RGB halo stars. The 100 Myr isochrone accounts for some of the fainter main sequence (MS) stars and most of horizontal branch core helium burners (HeB). The 30 Myr isochrone accounts for the brighter MS stars, HeBs, and red supergiants. We assume an [M/H]$= -$0.25 for the $\le500$ Myr isochrones. (*fig:m82-angst*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.10895"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

131  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
